# <span style="color:lightblue">**Estudo_Aplicado_Playlists:**
##### **Grupo: Caio, Gael, Renatão e Gabriel**

### **Tarefas da semana: 25/06**

1. **Baixar playlists do artofthemix (dados do paper)**
2. **Fast Text (Facebook) + Gensim**
3. **Executar o Fast Text ou Gensim nas playlists do artofthemix**
4. **Representar cada música como sendo artista-música**
    - Exemplo: The Beatles - Free As A Bird -> TheBeatles::FreeAsABird
5. **Transformar cada playlist em uma linha num arquivo de texto e rodar o FastText nesse arquivo**
6. **Ver alguns exemplos de similaridade**


# **Etapas🤖:**


## **Etapa_1 - Limpeza e Preparo 🧹:**


In [123]:
!pip install wheel
!pip install fasttext-wheel


In [125]:
import csv

In [126]:
!pip install pandas
!pip install numpy

In [127]:
import pandas as pd
import numpy as np

In [128]:
import json
import gzip
import pandas as pd
import re
import ast


# Abre e lê o arquivo JSON compactado
with gzip.open('aotm2011_playlists.json.gz', 'rt', encoding='utf-8') as playlists_file:
    playlists = json.load(playlists_file) 

# Converte a lista de playlists em um DataFrame do pandas
playlists_df = pd.DataFrame(playlists)

#Vamos contar quantas linhas possuimos do arquivo antes de realizar a limpeza e tratamento do dados 
print(playlists_df.shape)
#Imprimir dataframe
playlists_df


(101343, 6)


,category,filtered_lists,playlist,timestamp,mix_id,user
0,Mixed Genre,"[[SOFDPDC12A58A7D198], [SOPIEQP12A8C13F268, SO...","[[[peter murphy, marlene dietrich's favourite ...",2005-03-27T10:53:00,89567,"{'mixes_posted': '23', 'member_since': '2004-0..."
1,Theme,"[[SOQOVCB12A58A7E83B], [SOGKLYN12AB017F155, SO...","[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11T21:45:00,117124,"{'mixes_posted': '7', 'member_since': '2006-11..."
2,Theme,[[SOFIQBR12A6D4F78C6]],"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31T10:15:00,97418,"{'mixes_posted': '93', 'member_since': '2003-1..."
3,Theme,"[[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D], [SO...","[[[The Animals, San Francisco Nights], None], ...",2002-03-20T17:59:00,23876,"{'mixes_posted': '8', 'member_since': '2002-03..."
4,Theme,"[[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23], [SO...","[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23T03:43:00,70694,"{'mixes_posted': '60', 'member_since': '2004-0..."
...,...,...,...,...,...,...
101338,Hip Hop,"[[SODLSVK12A8C13EEE1], [SOEOMEJ12A8AE46299], [...","[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03T14:54:00,102561,"{'mixes_posted': '24', 'member_since': '2003-0..."
101339,Mixed Genre,"[[SONYEIJ12A6D4FA32D, SOSGUDA12A6D4FA2FB], [SO...","[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03T15:05:00,69379,"{'mixes_posted': '1', 'member_since': '2004-03..."
101340,Single Artist,[[SOYOFST12D0219B4BA]],"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18T00:01:00,18933,"{'mixes_posted': '23', 'member_since': '2001-1..."
101341,Theme,"[[SOKAWYU12A6D4F6769, SOXMBFV12A8C13B8DD], [SO...","[[[birthday party, big-jesus-trash-can], None]...",2006-01-28T10:16:00,100251,"{'mixes_posted': '40', 'member_since': '2002-0..."


In [146]:
#Função que remove caracteres:
def sub_preprocess(text):
    # Remove aspas simples e duplas
    text = re.sub(r'[\'"]', '', text)
    # Remove caracteres que não sejam letras, números ou espaços
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove múltiplos espaços e quebras de linha, substituindo por um único espaço
    text = re.sub(r'[ \n]+', ' ', text)
    return text.strip()

In [130]:
#Função que utiliza a função sub_preprocess e também realiza outras mudanças:
def preprocess(element):
    if isinstance(element, str):
        return sub_preprocess(element).lower()  # Converte para minúsculas após preprocess
    elif isinstance(element, list):
        # Converte lista aninhada em string e aplica preprocess
        return sub_preprocess(str(element)).lower()
    else:
        return element

In [145]:
#Exemplo:

import re

text = """[[['peter murphy', "marlene dietrich's favourite poem"], None],
 [['the walker brothers', "the sun ain't gonna shine anymore"],
  'SOFDPDC12A58A7D198'],
 [['marc almond', 'jacky'], None],
 [['tindersticks', 'dying slowly'], None],
 [['tori amos', 'me and a gun'], 'SOPIEQP12A8C13F268'],
 [['suzanne vega', 'luka'], 'SOKMCJK12A6D4F6105'],
 [['madonna', 'spanish eyes'], None],
 [['the angels of light', 'praise your name'], None],
 [['eurythmics', 'sex crime'], None],
 [['tom waits', 'drunk on the moon'], None],
 [['kate bush', 'wuthering heights'], 'SOGTGJR12A6310E08D'],
 [['david bowie', "new york's in love"], 'SOLTBYJ12A6310F2BB'],
 [['echo & the bunnymen', 'crocodiles'], 'SOBOXXN12A6D4FA1A2'],
 [['peter murphy', "i'll fall with your knife"], 'SOUQUFO12B0B80778E']]"""


In [132]:
#Testando exemplo
preprocess(text)

'peter murphy marlene dietrichs favourite poem none the walker brothers the sun aint gonna shine anymore sofdpdc12a58a7d198 marc almond jacky none tindersticks dying slowly none tori amos me and a gun sopieqp12a8c13f268 suzanne vega luka sokmcjk12a6d4f6105 madonna spanish eyes none the angels of light praise your name none eurythmics sex crime none tom waits drunk on the moon none kate bush wuthering heights sogtgjr12a6310e08d david bowie new yorks in love soltbyj12a6310f2bb echo the bunnymen crocodiles soboxxn12a6d4fa1a2 peter murphy ill fall with your knife souqufo12b0b80778e'

In [133]:
# Dropar colunas específicas do DaaFrame:
columns_to_drop = ['timestamp', 'user', 'mix_id', 'filtered_lists']  # Substitua pelos nomes das colunas que deseja remover
playlists_df = playlists_df.drop(columns=columns_to_drop)
playlists_df

,category,playlist
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ..."
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None..."
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ..."
3,Theme,"[[[The Animals, San Francisco Nights], None], ..."
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY..."
...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan..."
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte..."
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4..."
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]..."


In [147]:
# Defina as colunas nas quais você deseja aplicar a função
columns_to_process = ['playlist', 'category']

# #Aplicar função preprocess nas colunas:
for column in columns_to_process:
    playlists_df[column] = playlists_df[column].apply(preprocess)
    playlists_df[column] = playlists_df[column].apply(preprocess)

playlists_df

,category,playlist,playlist_description
0,labelmixed genre,peter murphy marlene dietrichs favourite poem ...,__label__mixed genre peter murphy marlene diet...
1,labeltheme,the cure edge of the deep green sea none jeff ...,__label__theme the cure edge of the deep green...
2,labeltheme,peter gabriel the feeling begins none ofra haz...,__label__theme peter gabriel the feeling begin...
3,labeltheme,the animals san francisco nights none the zomb...,__label__theme the animals san francisco night...
4,labeltheme,sparklehorse its a wonderful life soouytp12b0b...,__label__theme sparklehorse its a wonderful li...
...,...,...,...
101338,labelhip hop,r kelly burn it up remix feat daddy yankee sod...,__label__hip hop r kelly burn it up remix feat...
101339,labelmixed genre,pachelbel grave none prefuse73 afternoon love ...,__label__mixed genre pachelbel grave none pref...
101340,labelsingle artist,queen bohemian rhapsody soyofst12d0219b4ba tie...,__label__single artist queen bohemian rhapsody...
101341,labeltheme,birthday party bigjesustrashcan none xmal deut...,__label__theme birthday party bigjesustrashcan...


In [136]:
#Rotular dados para treinar um dos modelos de classificação:
playlists_df['category'] = '__label__' + playlists_df['category'].astype(str)
playlists_df['playlist_description'] = playlists_df['category'] + ' ' + playlists_df['playlist']
playlists_df.head()


,category,playlist,playlist_description
0,__label__mixed genre,peter murphy marlene dietrichs favourite poem ...,__label__mixed genre peter murphy marlene diet...
1,__label__theme,the cure edge of the deep green sea none jeff ...,__label__theme the cure edge of the deep green...
2,__label__theme,peter gabriel the feeling begins none ofra haz...,__label__theme peter gabriel the feeling begin...
3,__label__theme,the animals san francisco nights none the zomb...,__label__theme the animals san francisco night...
4,__label__theme,sparklehorse its a wonderful life soouytp12b0b...,__label__theme sparklehorse its a wonderful li...


## **Etapa_2 - Treinando os modelos 📖:**

In [151]:

import fasttext # Importa a função fasttext
from sklearn.model_selection import train_test_split  # Importa a função train_test_split do módulo sklearn.model_selection

train, test = train_test_split(playlists_df, test_size=0.2) # Divide o DataFrame playlists_df em conjuntos de treinamento (train) e teste (test)


# test_size=0.2 significa que 20% dos dados serão usados
#para o conjunto de teste e 80% para o conjunto de treinamento


In [149]:
#Obtém as dimensões dos DataFrames de treinamento e teste
train.shape, test.shape

((81074, 3), (20269, 3))

In [153]:
# Salva o DataFrame 'train' em um arquivo de texto chamado 'playlists.train.txt'
train.to_csv("playlists.train.txt", columns=["playlist_description"], index=False, header=False)

# Salva o DataFrame 'test' em um arquivo de texto chamado 'playlists.test.txt'
test.to_csv("playlists.test.txt", columns=["playlist_description"], index=False, header=False)

In [170]:
# Modelo Não Supervisionado, geração de (vetores de palavras).
model_1 = fasttext.train_unsupervised("playlists.train.txt")

In [172]:
# Modelo Supervisionado, utilizado para tarefas de classificação de texto, como categorizar textos ou prever rótulos:
model_2 = fasttext.train_supervised(input="playlists.train.txt")

In [ ]:
#Após treinar o modelo, nos podemos avaliá-lo usando o conjunto 'test'
results = model.test("playlists.test.txt")
print(f"Accuracy: {results.precision}")

## **Etapa_3 - Testando Métodos e Analisando Similaridade 🧪:**

##### **Métodos para Modelo Não Supervisionado (train_unsupervised)**


In [176]:
model_1.get_nearest_neighbors("samba", k=10)

[(0.7875746488571167, 'tamba'),
 (0.7800137996673584, 'gilberto'),
 (0.7680831551551819, 'soipxxx12a67add3e7'),
 (0.7582449316978455, 'bebel'),
 (0.7561060786247253, 'getzgilberto'),
 (0.7552695274353027, 'batucada'),
 (0.7503167986869812, 'bencao'),
 (0.7494335770606995, 'moraes'),
 (0.7456175684928894, 'ossanha'),
 (0.7413249015808105, 'cao')]

In [175]:
model_1.get_word_vector("metallica")

array([-0.26164058, -0.22267719, -0.11761403, -0.05819905,  0.68340063,
        0.2437857 ,  0.00393445,  0.76994157, -0.38434893,  0.00766789,
        0.02152312, -0.26596698,  0.3511865 ,  0.52563006, -0.29511562,
       -0.02673878,  0.08992292, -0.45407444, -0.01219232, -0.5718898 ,
       -0.34960434,  0.6989941 ,  0.72777027,  0.25769854, -0.04013168,
       -0.49677002, -0.11890286,  0.05661783, -0.0198641 , -0.53436863,
       -0.06318267,  0.2143906 ,  0.06095757,  0.1118056 ,  0.6185597 ,
       -0.6075136 , -0.42886686,  0.39774823, -0.03172579, -0.04666685,
        0.04168625,  0.18392767, -0.869164  , -0.4239288 , -0.13170563,
        0.15331088,  0.5871268 ,  0.37241516,  0.09936754,  0.78752434,
        0.23167543, -0.13547112, -0.11302575, -0.6408546 ,  0.27911255,
       -0.80732065, -0.71219903,  0.00863613,  0.16735408, -0.19064285,
        0.25408742, -0.7333515 ,  0.28009865, -0.48554695, -0.39660656,
        0.15511517,  0.12211005, -0.17238195,  0.51081467, -0.25

In [169]:
model_1.get_words(include_freq=True)

(['none',
  'the',
  'of',
  'you',
  'a',
  '</s>',
  'i',
  'in',
  'and',
  'to',
  'me',
  'love',
  'my',
  'on',
  'for',
  'genre',
  'is',
  'your',
  'it',
  'be',
  'dont',
  'all',
  'up',
  'with',
  'song',
  'new',
  'no',
  'girl',
  'day',
  'live',
  'get',
  'man',
  'like',
  'one',
  'from',
  'im',
  'this',
  'black',
  'world',
  'out',
  'down',
  'john',
  'time',
  'rock',
  'go',
  'little',
  'boys',
  'good',
  'what',
  'blue',
  'do',
  'life',
  'band',
  'we',
  'that',
  'beatles',
  'eyes',
  'its',
  'by',
  'big',
  'so',
  'at',
  'are',
  'baby',
  'boy',
  'night',
  'heart',
  'want',
  'white',
  'blues',
  'la',
  'bob',
  'dead',
  'not',
  'radiohead',
  'red',
  'ben',
  'david',
  'way',
  'will',
  'come',
  'when',
  'just',
  'back',
  'know',
  'tom',
  'hot',
  'bad',
  'here',
  'got',
  'if',
  'dj',
  'over',
  'death',
  'cant',
  'remix',
  'girls',
  'mix',
  'away',
  'can',
  'sun',
  'smith',
  'young',
  'never',
  'sweet',


##### **Métodos para Modelo Supervisionado (train_supervised)**


In [195]:
model_2.predict("jayz", k=4)

(('__label__electronic',
  '__label__dancehouse',
  '__label__world',
  '__label__jazz'),
 array([0.6799764 , 0.19278015, 0.05103604, 0.03519744]))

In [198]:
model_2.get_labels(include_freq=True)

(['__label__mixed',
  '__label__theme',
  '__label__single',
  '__label__rockpop',
  '__label__indie',
  '__label__alternating',
  '__label__romantic',
  '__label__road',
  '__label__hip',
  '__label__punk',
  '__label__depression',
  '__label__break',
  '__label__narrative',
  '__label__electronic',
  '__label__sleep',
  '__label__dancehouse',
  '__label__cover',
  '__label__country',
  '__label__rhythm',
  '__label__jazz',
  '__label__hardcore',
  '__label__folk',
  '__label__ambient',
  '__label__experimental',
  '__label__world',
  '__label__reggae',
  '__label__rock',
  '__label__industrial',
  '__label__blues',
  '__label__goth',
  '__label__avant',
  '__label__classical',
  '__label__drum',
  '__label__skatethrash',
  '__label__techno',
  '__label__exercise',
  '__label__microsound',
  '__label__drums',
  '__label__religiousspiritual'],
 array([37261, 11648,  5017,  4316,  4090,  4070,  2131,  1583,  1194,
         1079,   989,   901,   866,   692,   591,   559,   536,   384,
  

In [161]:
model.get_word_vector("metallica")

array([ 0.08684882, -0.03883178,  0.03447979,  0.21369083,  0.6375476 ,
        0.12851574, -0.28334635,  0.3518048 , -0.4608986 ,  0.00465115,
        0.215038  , -0.46904045, -0.01834383,  0.47778976,  0.0986895 ,
       -0.24159741, -0.02603383, -0.19385281, -0.00828294,  0.03835469,
        0.03297026,  0.90222234,  0.89991504,  0.19602557, -0.09361597,
       -0.18551186, -0.11557156, -0.08871364, -0.05092335, -0.36941272,
        0.09650524,  0.42190775, -0.20576617,  0.16693936,  0.6922783 ,
       -0.13196449, -0.80640775,  0.13245538, -0.34642383,  0.35749507,
        0.01549249,  0.41852   , -0.59903157, -0.13042472, -0.03055201,
        0.02054101,  0.71158636,  0.50128156,  0.4427316 ,  0.9325362 ,
       -0.03340987, -0.08488344, -0.4108567 , -0.77844083,  0.16352479,
       -0.5356394 , -0.3132883 , -0.27063984,  0.07568128, -0.3477124 ,
        0.56279784, -0.83100665, -0.12823083, -0.63693106, -0.4140135 ,
        0.49470192, -0.12899703, -0.3817528 ,  0.23778188, -0.31

## **Etapa_4 - Duvidas, Curiosidades e Finalização 💡:**

##### **Duvidas como:**


1- **Como utilizar o modelo supervisonado da forma adequada?**

2- **Processo de Clusterização**

3- **Formas para melhorar nosso modelo?**

4- **Funcinamento do _label_ na hora de rotular?**


### **Obrigado pela Atenção 👍**
